*Note 1: This is the 2015-16 dataset. Lot of things might have been changed by now*

This notebook contains a short analysis of the literacy rate in Indian states and how it depends on the state's education indicators. I will be following CRISP-DM and will be marking my steps throughout the notebook.
The steps for CRISP-DM are as follows: 

1. Business understanding: outlining the questions we will answer along with why they are relevant and important
2. Data understanding: understanding the data and providing stats about it
3. Data preparation: showing how we clean and prepare the data
4. Data modeling: if we created a model, this is the section we would put it in of the analysis
5. Results evaluation: conclusion and discussion


In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [ ]:
# Reading in datasets
elementary = pd.read_csv('../input/2015_16_Statewise_Elementary.csv')
meta_elementary = pd.read_csv('../input/2015_16_Statewise_Elementary_Metadata.csv')

Some functions that have been used throughout the notebook have been defined here:

## Defining functions

In [ ]:
def sort_vals(df,col):
    '''
    sorts values according to the column provided in ascending order and returns a dataframe with 
    index reset
    
    INPUT:
    df - dataframe to be sorted
    col - column to be sorted on
    
    OUTPUT:
    df_new - sorted dataframe with index reset
    
    '''
    df_new = df.sort_values(by = col)
    df_new.reset_index().drop('index', axis = 1, inplace = True)
    return df_new

In [ ]:
# We could just have used df.plot.barh method instead of the below. Both of these have the same output
def plot_barh(df, col1, col2, size = (10,10), title = None, xlabel = None):
    '''
    to make a horizontal barpot using any two columns(x and y axis) from a dataframe
    
    INPUT:
    df - dataframe that contains the data
    col1 - column for y-axis
    col2 - column for x-axis
    size - figsize attribute of the plot
    title - title of the plot
    xlabel - x-axis label of the plot
    
    '''
    fig, ax = plt.subplots(1,1,figsize = size)
    plt.barh(df[col1],df[col2],label = col2)
            
    if title:
        ax.set_title(title)   
    else:
        ax.set_title('{} vs {}'.format(col2, col1))
    if xlabel:
        ax.set_xlabel(xlabel)
    else:
        ax.set_xlabel(col2)

In [ ]:
def display_desc(row):
    '''
    Displays the full string in the description column of the meta elementary dataset
    
    INPUT:
    row - (str) value of the field name you want the description for
    
    '''
    return meta_elementary[meta_elementary['Field name'] == row].Description.values[0]

## Exploring the data

Here, we are just taking a small look at the data to understand how to go about it working with it

In [ ]:
# Checking the shape of the datasets
print(elementary.shape)
print(meta_elementary.shape)

In [ ]:
# Quick check for null entries
print(elementary.isnull().sum()[elementary.isnull().sum() > 0].shape)

In [ ]:
elementary.head(2)

In [ ]:
meta_elementary.head(2)

We see that:
* elementary contains the different states along with their education indicator values.
* meta_elementary gives us the description of the various labels used in elementary.

### [Step 1] Business Understanding:

We will be trying to answer the following questions : 
1. Which states have the highest and the lowest literacy rates? 
2. How are the top 3 states different from the bottom 3 states and what factors can the bottom 3 states work on to increase their literacy rate?
3. Which class is the dropout rate maximum in ?

These questions help us not only in determining which states are the top and bottom 3 w.r.t. to overall literacy rate, but also in studying about factors that affect these states. The 3rd question is really important from the governments view as it gives us the class/age of the child where he/she is most likely to dropout and thus, that particluar age group can be targeted with highly personal and specialized incentives.

### [Step 2] Data Understanding:

Here will be trying to get a little better look at the data in elementary.

In [ ]:
# The overall state literacy rate
elementary['OVERALL_LI'].describe()

In [ ]:
elementary.AREA_SQKM.describe()

In [ ]:
elementary.GROWTHRATE.describe()

In [ ]:
elementary[elementary.GROWTHRATE == -0.47]

Its interesting to see that Nagaland had a negative growth rate for 2015-16

In [ ]:
# difference between male and female literacy rates for each state
(elementary.MALE_LIT - elementary.FEMALE_LIT).describe()

The standard deviation is pretty large for this and we see that that 50% of the states have difference in male and female literacy rate as more than 13.72 which to me, seems pretty high !

#### Let's go onto answer our questions now that we have got some insights about our data.

_Note : As data is clean, the 3rd step in CRISP-DM, data preparation will be written multiple times when we try to create new features_

## 1. Which states have the highest and lowest literacy rates ? 

*NOTE: Secondary and elementary have the same stats concerning literacy rates.*

In [ ]:
# 'OVERALL_LI' is the overall literacy rate for each state
elementary = sort_vals(elementary, 'OVERALL_LI')

In [ ]:
plot_barh(elementary, 'STATNAME', 'OVERALL_LI', title = 'Literacy rates comparison \
between different states' , xlabel = 'Literacy rate')

In [ ]:
elementary = sort_vals(elementary, 'FEMALE_LIT')

In [ ]:
plot_barh(elementary, 'STATNAME', 'FEMALE_LIT')

In [ ]:
elementary = sort_vals(elementary, 'MALE_LIT')

In [ ]:
plot_barh(elementary, 'STATNAME', 'MALE_LIT')

We will now see a Male vs Female vs Literacy rate comparison

In [ ]:
elementary = sort_vals(elementary, 'OVERALL_LI')

In [ ]:
elementary.plot(kind = 'bar',x = 'STATNAME',y = ['MALE_LIT','FEMALE_LIT'],stacked = False,figsize=(12,6),width = 0.7,color = ['salmon','dimgrey'])

There are two things that could be interesting to explore:

1. Make a new column showing the the differences in literacy rates between men and women.
2. It looks like North-Eastern states on an average have less difference between male and female literacy rates. 

### [Step 3] Data Preparation:

Here we are going to create a new feature, DIFF_LIT, by subtracting the Female literacy rate from the male literacy rate. This will help us see the difference in male and female literacy rate for each state easily.

In [ ]:
# Difference in Male and Female literacy rates
elementary['DIFF_LIT'] = elementary.MALE_LIT - elementary.FEMALE_LIT 

In [ ]:
# Average difference in literacy rates
diff_lit_mean = elementary.DIFF_LIT.mean()
print("India's avg difference in literacy rate between men and women is : {}".format(diff_lit_mean))

In [ ]:
# States with the least male and female literacy rates difference
sort_vals(elementary, 'DIFF_LIT').head()[['STATNAME','DIFF_LIT']]

In [ ]:
# States with the most male and female literacy rates difference
sort_vals(elementary, 'DIFF_LIT').tail()[['STATNAME','DIFF_LIT']]

We now check out how the North-East Indian states perform compared to the National Average

In [ ]:
# These are the Indian North-East states
north_east = ['NAGALAND','MANIPUR','MIZORAM','ASSAM','TRIPURA','ARUNACHAL PRADESH','MEGHALAYA','SIKKIM']
elementary.set_index(elementary.STATNAME,inplace = True)
ne_diff_lit_mean = elementary.loc[north_east,'DIFF_LIT' ].mean()
print('The avg in DIFF_LIT for north-eastern states ({}) is much less than the national avg ({}).'.format(ne_diff_lit_mean,diff_lit_mean))

In [ ]:
print("The Female literacy rate for meghalaya : {} .VS. the avg female literacy rate : {}".format(elementary.loc['MEGHALAYA','FEMALE_LIT'],elementary.FEMALE_LIT.mean()))

We note the following from the above :

* The states with the highest overall literacy rates are : Kerala, Lakshwadeep, Mizoram, Tripura and Goa.
* The states with the lowest overall literacy rates are : Bihar, Telngana, Arunachal Pradesh, Rajasthan and Jharkhand. 
* Dadra and Nagar Haveli, Rajasthan, Chattisgarh, Jharkhand and MP have the maximum difference between Male and Female literacy rates. 
* Though the top states in male and female literacy remain the same but when we see the bottom states, they change. Meghalaya has the 5thhighest no. of illiterate males. The avg female literacy rate for Meghalaya though is higher than the avg female_lit rate.
* Also, the states with the highest overall literacy rates have really little differences between male and female literacy rates in general. We also see that states in North east have on an average lesser difference between male and female literacy rates when compared with the average of the country.

## 2. How are the top 3 states different from the bottom 3 states and what factors can the bottom 3 states work on to increase their literacy rate ? ( Explore using elementary dataset only)

For answering this question, we will carry out visual analyses using bar graphs on different features to see how they can affect Overall Literacy Rate

### [Step 3] Data Preparation:

We will be creating top_bottom, a dataframe that contains only the top 3 and bottom 3 states w.r.t. overal literacy rate. This will make plotting and analysis easy for us as we just have to explore these 6 states for answering this question.

In [ ]:
# We will drop Telangana from bottom 3 as its been recently founded(2014)
top_3_elem = elementary.sort_values(by = 'OVERALL_LI', ascending = False).head(3)
bottom_3_elem = elementary.sort_values(by = 'OVERALL_LI', ascending = True).head(4).drop('TELANGANA',axis = 0)

In [ ]:
top_bottom = pd.concat([top_3_elem, bottom_3_elem], axis = 0, sort = False)

In [ ]:
top_bottom

Now we will go over different features that we think affect our state's overall literacy rate

1. TOTPOPULAT

In [ ]:
display_desc('TOTPOPULAT')

In [ ]:
top_bottom.TOTPOPULAT/top_bottom.AREA_SQKM * 1000

So population density isn't a factor at all in differentiating between the above two groups. At first I thought that maybe having higher population density created more pressure on the govt to cater to the needs of the people and that it might be difficult for states of really high population density to do so.But the govt of that state should be well equipped enough to make sure that people get access to education in that state irrelevent of how dense the population is.

2. DIFF_LIT

DIFF_LIT is the variable we created earlier that  by subtracting female literacy rate from the male literacy rate

In [ ]:
top_bottom.DIFF_LIT

In [ ]:
top_bottom.plot(y = 'DIFF_LIT',kind= 'bar')
plt.title('Top 3 vs Bottom 3 states for DIFF_LIT')

DIFF_LIT says a lot. The differences are really high in states with low overall literacy rates. So even if the bottom most states have good male literacy rates, female literacy rates are really low and that takes their overall literacy rate down. Thus these states really need to work on educating their females and increasing their literacy rate.

3. P_URB_POP

### [Step 3] Data Preparation:

Here will create `P_RUR_POP` which is 100 - urban population proportion. This will help us in comparing the rural and urban population proportions.

In [ ]:
display_desc('P_URB_POP')

In [ ]:
top_bottom['P_RUR_POP'] = 100 - top_bottom['P_URB_POP']

In [ ]:
top_bottom.plot(y = ['P_URB_POP', 'P_RUR_POP'], kind = 'bar', figsize = (10,5))

Here we see a pretty obvious but important component that differentiates our top 3 and bottom 3 states. The difference between rural and urban population is much much bigger in the bottom 3.

The rural population percentage in the bottom 3 states is much more than the rural population percentage in the top 3. That's an important factor to note. People living in rural areas lead a very different life compared to the people living in urban areas. There is less motivation to go to school in rural areas as a lot of people tend to take up their parents profession or business. Also, often children are made to skip school and work at the farm.

4. SEXRATIO

Sex ratio is the no. of females per 1000 males

In [ ]:
# Approximate National average 
elementary.SEXRATIO.mean()

In [ ]:
top_bottom.SEXRATIO

In [ ]:
top_bottom.plot(y ='SEXRATIO', kind = 'bar')

In [ ]:
# Lets include more states in the comparison (considering the top and bottom 10 states)
elementary.sort_values(by = 'OVERALL_LI', inplace=True)
pd.concat([elementary.head(10),elementary.tail(10)], axis = 1, sort = False).plot(kind= 'bar', y = 'SEXRATIO', figsize = (10,7),width = 1)

In [ ]:
elementary.loc[:,['SEXRATIO','OVERALL_LI']].corr()

Clearly the sex ratio has nothing to do with the literacy level.The graphs dont show sex ratio affecting the literacy rate. Also the correlation's too weak. 

5. P_SC_POP and P_ST_POP

In [ ]:
print(display_desc('P_SC_POP'))
print(display_desc('P_ST_POP'))

In [ ]:
top_bottom['SC_ST_POP'] = top_bottom.P_SC_POP + top_bottom.P_ST_POP

In [ ]:
top_bottom.plot(y = ['SC_ST_POP','P_SC_POP','P_ST_POP'], kind = 'bar', figsize = (15,7))

Clearly, the overall literacy rate doesn't depend on the SC and ST population proportions because these are very different for all the above states.

Now we will try to analyse some school related features and see if they affect the overall literacy rate.

In [ ]:
# These are all the 'school total' related variables that we can explore
meta_sch = meta_elementary[meta_elementary['Field name'].str.contains(r'^SCH\w*TOT')]
print(meta_sch)

6. KIDSPERSCH (Kids per school)(For primary/elementary schools)

### [Step 3] Data Preparation:

We will be making `KIDSPERSCH` using `SCHKIDS`, `TOT_6_10_15` and `TOT_11_13_15` as this will let us check if the number of kids per school is a factor that determines the overall literacy rate or not.

In [ ]:
display_desc('SCHTOT')

In [ ]:
display_desc('TOT_6_10_15')

In [ ]:
display_desc('TOT_11_13_15')

In [ ]:
# Total no. of schools for the top 3 and bottom 3 states 
top_bottom.SCHTOT

In [ ]:
# No. of kids in the age group 6-15, i.e, elementary schoolkids
top_bottom['SCHKIDS'] = top_bottom.TOT_6_10_15 + top_bottom.TOT_11_13_15
# No. of kids per school
top_bottom['KIDSPERSCH'] = top_bottom.SCHKIDS/top_bottom.SCHTOT

In [ ]:
top_bottom.plot(y = 'KIDSPERSCH', kind = 'bar',)

We see that the no. of kids per school is not a good measure to differentiate between the top and bottom 3 states. Though Mizoram has has the lowest number of kids per school, Arunachal Pradesh and Rajasthan are closeby. Bihar has the max no. of kids per school. 

One reason why this measure failed to differentiate could be that we dont know the size of the schools and thus there could be really big schools in Kerala but really small schools (generally speaking) in Arunachal Pradesh. Thus we don't get to know the no. of students each school can enroll and manage effeciently. No. of teachers is too a huge factor here. Perhaps no. of classes or sections would have been a better measure here as we would have gotten to know the average class strength.

7. TOTCLS1g and KIDSPERCL

### [Step 3] Data Preparation:

We will making `KIDSPERCL` (Kids per class) as we think that it might be a better indicator of the overall literacy rate of the state.

In [ ]:
display_desc('TOTCLS1G')

In [ ]:
top_bottom['TOTCLS1G'].plot(kind= 'bar')

In [ ]:
# primary kids per class
top_bottom['KIDSPERCL'] = top_bottom['SCHKIDS']/top_bottom['TOTCLS1G']

In [ ]:
top_bottom['KIDSPERCL'].plot(kind= 'bar')

In [ ]:
elem = elementary.copy(deep = True)
elem['SCHKIDS'] = elem.TOT_6_10_15 + elem.TOT_11_13_15
elem['KIDSPERCL'] = elem['SCHKIDS']/elem['TOTCLS1G']
elem.sort_values(by = 'OVERALL_LI',ascending = False,inplace = True)

In [ ]:
pd.concat([elem.head(5),elem.tail(5)], axis = 0, sort = False).plot(kind= 'bar', y = 'KIDSPERCL', figsize = (10,7),width = 1)

In [ ]:
elem[['KIDSPERCL','OVERALL_LI']].corr()

We see that there is a difference in the no. of kids per class, but it is nt significant.Though Bihar and Rajasthan have a very high no. of primary kids per class. The top 5 states on an average have lesser kids per class when compared to the bottom 5 states with the exception of Arunachal Pradesh.

8. Comparing the percentage of different types of Primary schools

### [Step 3] Data Preparation:

We can also take a look at the different types of schools in these states. It might be that the states having a high literacy rate have more private schools than govt. schools. For this purpose, we do some data preparation to allow for plotting of the type of schools.

In [ ]:
# Plot of the % of elementary private, govt schools and Madarsas and comparing them with the national avg.
schtotg_avg = (elementary.SCHTOTG/elementary.SCHTOT).mean()
schtotp_avg = (elementary.SCHTOTP/elementary.SCHTOT).mean()
schtotm_avg = (elementary.SCHTOTM/elementary.SCHTOT).mean()

top_3_elem['SCHTOTG_P'] = (top_3_elem.SCHTOTG/top_3_elem.SCHTOT)
top_3_elem['SCHTOTP_P'] = (top_3_elem.SCHTOTP/top_3_elem.SCHTOT)
top_3_elem['SCHTOTM_P'] = (top_3_elem.SCHTOTM/top_3_elem.SCHTOT)

bottom_3_elem['SCHTOTG_P'] = (bottom_3_elem.SCHTOTG/bottom_3_elem.SCHTOT)
bottom_3_elem['SCHTOTP_P'] = (bottom_3_elem.SCHTOTP/bottom_3_elem.SCHTOT)
bottom_3_elem['SCHTOTM_P'] = (bottom_3_elem.SCHTOTM/bottom_3_elem.SCHTOT)

pd.concat([top_3_elem, bottom_3_elem], axis = 1, sort = False).plot(y = ['SCHTOTP_P','SCHTOTG_P','SCHTOTM_P'],\
                                                                    figsize = (18,8), kind = 'bar',\
                                                                    color =['blue','blue','g','g','black','black'], \
                                                                       )

plt.plot([schtotg_avg]*8, linestyle ='--',color = 'g') # National avg for % of govt schools

plt.plot([schtotp_avg]*8, linestyle ='--',color = 'blue') # National avg for % of private schools

plt.plot([schtotm_avg]*8, linestyle ='--',color= 'black') # National avg for % of Madarsas

plt.legend(['Pvt','Govt','Madarsas'],loc = 1)

From here we can clearly note the following: 
* Lakshadweep has only govt schools.
* Kerala has the highest number of Pvt schools share in the total schools and is the only state here which has more Pvt schools than Govt schools. It has also got an exceptionally high number of Madarasa and the number of govt schools is much less than the national average.
* Bihar and Arunachal Pradesh have really less pvt schools compared to the govt schools. Their % of govt school is more than the national avg.
* Rajasthan has around 35% pvt schools which is large compared to its no. of govt schools.

All of this just shows us that even the share of pvt and govt schools isnt related to the literacy rate.

9. CONTIE

In [ ]:
display_desc('CONTIE')

In [ ]:
elem[['CONTIE','OVERALL_LI']].corr()

In [ ]:
top_bottom.CONTIE.plot(kind = 'bar')

The above says that the more the overall literacy level is, the lesser the grants would have been given to the states for school development. That does make sense as states with lesser education standards and literacy level would be requiring lesser funds as they already have good infrastructure in place. It also depends on how many schools the state has or dense the population is. Lakshadweep, Mizoram and Arunachal Pradesh all are small states and thus their fund are minimal compared to Bihar and Rajasthans.

10. C9_B, C9_G, C8_B and C8_G

### [Step 3] Data Preparation:

Finally, here we calculate the dropout rate from class 8th to 9th. For that, we first sum up `C9_G` and `C9_B` to calculate the total no. of enrollments for class 9th. We do the same thing for class 8th. Then we subtract the total enrollments for class 9th from class 8th and divide this by the total enrollments for class 8th  to find out the proportion of students who dropped out from 8th to 9th

In [ ]:
display_desc('C9_B')

In [ ]:
display_desc('C9_G')

In [ ]:
display_desc('C8_B')

In [ ]:
display_desc('C8_G')

We can use the above 4 features to calculate the percentage of children dropping out in top 3 and bottom 3 states from class 8th to class 9th (primary to secondary).

In [ ]:
elementary['C9_TOTAL'] = elementary.loc[:,'C9_B'] + elementary.loc[:,'C9_G']

In [ ]:
elementary['C8_TOTAL'] = elementary.loc[:,'C8_B'] + elementary.loc[:,'C8_G']

In [ ]:
elementary['DROP_8_9'] = (elementary.C8_TOTAL - elementary.C9_TOTAL)/elementary.C8_TOTAL

In [ ]:
top_bottom_drop = pd.concat([elementary.sort_values('OVERALL_LI').tail(3).DROP_8_9, 
                             elementary.sort_values('OVERALL_LI').head(3).DROP_8_9], axis = 0)

In [ ]:
top_bottom_drop

In [ ]:
top_bottom_drop.plot(kind='bar', y = 'DROP_8_9', x = 'STATNAME')

In [ ]:
elementary[['DROP_8_9','OVERALL_LI']].corr()

We find the drop out rate from 8th to 9th(primary to secondary) a pretty important feature in distinguishing between the top 3 and bottom 3 states. It is an obvious reason but no one knew that there would be this much of a difference. The top 3 states have more admissions in 9th than they had in 8th while the bottom 3 have the opposite going on. 

Thus, the key takeaways would be from this analysis would be :

* States should try to increase their female literacy rate by  making it safer for girls to travel, making girl only schools with female teachers, providing special incentives to the families to get their girl child to omplete her schooling etc.
* People living in rural areas lead a very different life compared to the people living in urban areas. There is less motivation to go to school in rural areas as a lot of people tend to take up their parent's profession or business. Also, often children are made to skip school and work at the farm. The state should provide more incentives to ensure that families living in rural areas get their children to complete their schooling.
* Dropout rate from 8th to 9th class should be decreased and schemes should be made to ensure that this dropout rate declines. Causes for this dropout rate should be looked into and be dealt with.

## 3. Which class is the dropout rate maximum in ?

### [Step 3] Data Preparation:

To answer our final question, we must calculate the dropout rates for all classes given in the dataset. So we start by summing up the boys and the girls enrollment for each class. We form a dataframe out of this and then take the sum for each class, i.e, sum up total enrollments for all states. Then we calculate the dropout rate for each class and plot it

In [ ]:
# setting the index as state name for easy coding
elementary.set_index('STATNAME', inplace = True)

In [ ]:
# class_columns contains all the columns related to class enrollments
class_b_columns = elementary.columns[elementary.columns.str.contains('C\d_[B]$',case = False, regex = True)].tolist()
class_g_columns = elementary.columns[elementary.columns.str.contains('C\d_[G]$',case = False, regex = True)].tolist()

In [ ]:
# total enrollments in each class
sums_classes_df = pd.DataFrame(np.matrix(elementary[class_b_columns]) + np.matrix(elementary[class_g_columns]))

In [ ]:
# storing the column names and index for above dataframe
sums_classes_cols=[i[:2] for i in class_b_columns]
sums_classes_index = elementary.index

In [ ]:
sums_classes_df.index = sums_classes_index
sums_classes_df.columns = sums_classes_cols

In [ ]:
# totalling for columns
totals = sums_classes_df.sum(axis = 0)

In [ ]:
dropout = []
for i,_ in enumerate(totals):
    try: 
        dropout.append((totals[i] - totals[i+1])/totals[i])
    except:
        pass

In [ ]:
dropout

In [ ]:
plt.bar(x = range(2,10,1),height = dropout)

We see from this given graph that the dropout rate was maximum in the 6th class followed by the 9th class and 2nd class. Class 4 saw a negative dropout rate, i.e, more students enrolled and less dropped out when moving from class 3 to class 4 and that is some really good news !

The dropout rate for 5th to 6th was really high, 8.5%. One reason for this could be that when the child is old enough to be in 5th, he is usually old enough to do a lot of other work too. His parents might need him at the farm or at their business to help them and start earning. This is a huge problem, especially in the rural parts of India. An additional step here could be to compare the dropout rates between the rural and urban schools.

### We don't have a [Step 4] becuase we haven't modelled our data.

### [Step 5] Results and Conclusions :

We started out with a comparison of the literacy rates in the different states and union territories in India. We compared the Male and Female literacy rates in these states and saw that there was a huge difference in the states with a low overall literacy rate. The top 3 states were Kerala, Lakshadweep and Mizoram and the bottom 3 were Bihar, Telngana and Arunachal Pradesh.

Next we saw that difference in male and female literacy rates, rural population proportion and dropout rates from 8th to 9th class played a huge role in separating the top 3 and bottom 3 states.

Finally, the dropout rates in different classes were explored and while the dropout rate for 6th class was really high, more students had enrolled in class 4 than had dropped out.

The literacy rate in India has been improving but there are some key issues that need to be tackled aggressively in order to improve the state of education in India. This is not just the job of the govt, but it is the duty of each and everyone living in the country. Hope to see the stats showing a much better India in the future.

## Yay. Done !! 